In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]

In [2]:
alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])

In [3]:
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [6]:
with open('query_dict/id_querry_clean.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)

In [7]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [8]:
vals = all_groups['DocumentId'].values

In [9]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)
with open("url_simple.pickle", 'rb') as file:
    url_simple = pickle.load(file)
with open("host_simple.pickle", 'rb') as file:
    host_simple = pickle.load(file)

In [10]:
click_feats = dict()
click_feats1 = dict()
click_feats2 = dict()
click_feats['url_shows'] = []
click_feats['url_clicks'] = []
click_feats['url_ctr'] = []

click_feats['url_showpos_1'] = []
click_feats['url_showpos_2'] = []
click_feats['url_showpos_3'] = []
click_feats['url_showpos_4'] = []
click_feats['url_showpos_5'] = []
click_feats['url_showpos_6'] = []
click_feats['url_showpos_7'] = []
click_feats['url_showpos_8'] = []
click_feats['url_showpos_9'] = []
click_feats['url_showpos_10'] = []

click_feats['url_clickpos_1'] = []
click_feats['url_clickpos_2'] = []
click_feats['url_clickpos_3'] = []
click_feats['url_clickpos_4'] = []
click_feats['url_clickpos_5'] = []
click_feats['url_clickpos_6'] = []
click_feats['url_clickpos_7'] = []
click_feats['url_clickpos_8'] = []
click_feats['url_clickpos_9'] = []
click_feats['url_clickpos_10'] = []

click_feats1['url_ctrpos_1'] = []
click_feats1['url_ctrpos_2'] = []
click_feats1['url_ctrpos_3'] = []
click_feats1['url_ctrpos_4'] = []
click_feats1['url_ctrpos_5'] = []
click_feats1['url_ctrpos_6'] = []
click_feats1['url_ctrpos_7'] = []
click_feats1['url_ctrpos_8'] = []
click_feats1['url_ctrpos_9'] = []
click_feats1['url_ctrpos_10'] = []

click_feats1['url_pbm'] = []
click_feats1['url_time'] = []
click_feats1['url_avg_time'] = []

click_feats['host_shows'] = []
click_feats['host_clicks'] = []
click_feats['host_ctr'] = []

click_feats2['host_showpos_1'] = []
click_feats2['host_showpos_2'] = []
click_feats2['host_showpos_3'] = []
click_feats2['host_showpos_4'] = []
click_feats2['host_showpos_5'] = []
click_feats2['host_showpos_6'] = []
click_feats2['host_showpos_7'] = []
click_feats2['host_showpos_8'] = []
click_feats2['host_showpos_9'] = []
click_feats2['host_showpos_10'] = []

click_feats2['host_clickpos_1'] = []
click_feats2['host_clickpos_2'] = []
click_feats2['host_clickpos_3'] = []
click_feats2['host_clickpos_4'] = []
click_feats2['host_clickpos_5'] = []
click_feats2['host_clickpos_6'] = []
click_feats2['host_clickpos_7'] = []
click_feats2['host_clickpos_8'] = []
click_feats2['host_clickpos_9'] = []
click_feats2['host_clickpos_10'] = []

click_feats1['host_ctrpos_1'] = []
click_feats1['host_ctrpos_2'] = []
click_feats1['host_ctrpos_3'] = []
click_feats1['host_ctrpos_4'] = []
click_feats1['host_ctrpos_5'] = []
click_feats1['host_ctrpos_6'] = []
click_feats1['host_ctrpos_7'] = []
click_feats1['host_ctrpos_8'] = []
click_feats1['host_ctrpos_9'] = []
click_feats1['host_ctrpos_10'] = []

click_feats1['host_pbm'] = []
click_feats1['host_time'] = []
click_feats1['host_avg_time'] = []

In [11]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    click_feats['url_shows'].append(url_simple[key1].total_shows)
    click_feats['url_clicks'].append(url_simple[key1].total_clicks)
    if url_simple[key1].total_shows==0:
        click_feats['url_ctr'].append(0.0)
    else:
        click_feats['url_ctr'].append(url_simple[key1].total_clicks/url_simple[key1].total_shows)
    click_feats['url_showpos_1'].append(url_simple[key1].show_poses[0])
    click_feats['url_showpos_2'].append(url_simple[key1].show_poses[1])
    click_feats['url_showpos_3'].append(url_simple[key1].show_poses[2])
    click_feats['url_showpos_4'].append(url_simple[key1].show_poses[3])
    click_feats['url_showpos_5'].append(url_simple[key1].show_poses[4])
    click_feats['url_showpos_6'].append(url_simple[key1].show_poses[5])
    click_feats['url_showpos_7'].append(url_simple[key1].show_poses[6])
    click_feats['url_showpos_8'].append(url_simple[key1].show_poses[7])
    click_feats['url_showpos_9'].append(url_simple[key1].show_poses[8])
    click_feats['url_showpos_10'].append(url_simple[key1].show_poses[9])
    
    click_feats['url_clickpos_1'].append(url_simple[key1].click_poses[0])
    click_feats['url_clickpos_2'].append(url_simple[key1].click_poses[1])
    click_feats['url_clickpos_3'].append(url_simple[key1].click_poses[2])
    click_feats['url_clickpos_4'].append(url_simple[key1].click_poses[3])
    click_feats['url_clickpos_5'].append(url_simple[key1].click_poses[4])
    click_feats['url_clickpos_6'].append(url_simple[key1].click_poses[5])
    click_feats['url_clickpos_7'].append(url_simple[key1].click_poses[6])
    click_feats['url_clickpos_8'].append(url_simple[key1].click_poses[7])
    click_feats['url_clickpos_9'].append(url_simple[key1].click_poses[8])
    click_feats['url_clickpos_10'].append(url_simple[key1].click_poses[9])
    
    click_feats1['url_ctrpos_1'].append(url_simple[key1].click_poses[0]/(0.0000001+url_simple[key1].show_poses[0]))
    click_feats1['url_ctrpos_2'].append(url_simple[key1].click_poses[1]/(0.0000001+url_simple[key1].show_poses[1]))
    click_feats1['url_ctrpos_3'].append(url_simple[key1].click_poses[2]/(0.0000001+url_simple[key1].show_poses[2]))
    click_feats1['url_ctrpos_4'].append(url_simple[key1].click_poses[3]/(0.0000001+url_simple[key1].show_poses[3]))
    click_feats1['url_ctrpos_5'].append(url_simple[key1].click_poses[4]/(0.0000001+url_simple[key1].show_poses[4]))
    click_feats1['url_ctrpos_6'].append(url_simple[key1].click_poses[5]/(0.0000001+url_simple[key1].show_poses[5]))
    click_feats1['url_ctrpos_7'].append(url_simple[key1].click_poses[6]/(0.0000001+url_simple[key1].show_poses[6]))
    click_feats1['url_ctrpos_8'].append(url_simple[key1].click_poses[7]/(0.0000001+url_simple[key1].show_poses[7]))
    click_feats1['url_ctrpos_9'].append(url_simple[key1].click_poses[8]/(0.0000001+url_simple[key1].show_poses[8]))
    click_feats1['url_ctrpos_10'].append(url_simple[key1].click_poses[9]/(0.0000001+url_simple[key1].show_poses[9]))
    
    click_feats1['url_pbm'].append(url_simple[key1].total_clicks/(np.sum(url_simple[key1].show_poses*alpha)+0.0000001))
    click_feats1['url_time'].append(url_simple[key1].time)
    click_feats1['url_avg_time'].append(url_simple[key1].time/(url_simple[key1].total_clicks+0.0000001))

100%|██████████| 606050/606050 [01:04<00:00, 9379.21it/s] 


In [12]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    click_feats['host_shows'].append(host_simple[key1].total_shows)
    click_feats['host_clicks'].append(host_simple[key1].total_clicks)
    if host_simple[key1].total_shows==0:
        click_feats['host_ctr'].append(0.0)
    else:
        click_feats['host_ctr'].append(host_simple[key1].total_clicks/host_simple[key1].total_shows)
    click_feats2['host_showpos_1'].append(host_simple[key1].show_poses[0])
    click_feats2['host_showpos_2'].append(host_simple[key1].show_poses[1])
    click_feats2['host_showpos_3'].append(host_simple[key1].show_poses[2])
    click_feats2['host_showpos_4'].append(host_simple[key1].show_poses[3])
    click_feats2['host_showpos_5'].append(host_simple[key1].show_poses[4])
    click_feats2['host_showpos_6'].append(host_simple[key1].show_poses[5])
    click_feats2['host_showpos_7'].append(host_simple[key1].show_poses[6])
    click_feats2['host_showpos_8'].append(host_simple[key1].show_poses[7])
    click_feats2['host_showpos_9'].append(host_simple[key1].show_poses[8])
    click_feats2['host_showpos_10'].append(host_simple[key1].show_poses[9])
    
    click_feats2['host_clickpos_1'].append(host_simple[key1].click_poses[0])
    click_feats2['host_clickpos_2'].append(host_simple[key1].click_poses[1])
    click_feats2['host_clickpos_3'].append(host_simple[key1].click_poses[2])
    click_feats2['host_clickpos_4'].append(host_simple[key1].click_poses[3])
    click_feats2['host_clickpos_5'].append(host_simple[key1].click_poses[4])
    click_feats2['host_clickpos_6'].append(host_simple[key1].click_poses[5])
    click_feats2['host_clickpos_7'].append(host_simple[key1].click_poses[6])
    click_feats2['host_clickpos_8'].append(host_simple[key1].click_poses[7])
    click_feats2['host_clickpos_9'].append(host_simple[key1].click_poses[8])
    click_feats2['host_clickpos_10'].append(host_simple[key1].click_poses[9])
    
    click_feats1['host_ctrpos_1'].append(host_simple[key1].click_poses[0]/(0.0000001+host_simple[key1].show_poses[0]))
    click_feats1['host_ctrpos_2'].append(host_simple[key1].click_poses[1]/(0.0000001+host_simple[key1].show_poses[1]))
    click_feats1['host_ctrpos_3'].append(host_simple[key1].click_poses[2]/(0.0000001+host_simple[key1].show_poses[2]))
    click_feats1['host_ctrpos_4'].append(host_simple[key1].click_poses[3]/(0.0000001+host_simple[key1].show_poses[3]))
    click_feats1['host_ctrpos_5'].append(host_simple[key1].click_poses[4]/(0.0000001+host_simple[key1].show_poses[4]))
    click_feats1['host_ctrpos_6'].append(host_simple[key1].click_poses[5]/(0.0000001+host_simple[key1].show_poses[5]))
    click_feats1['host_ctrpos_7'].append(host_simple[key1].click_poses[6]/(0.0000001+host_simple[key1].show_poses[6]))
    click_feats1['host_ctrpos_8'].append(host_simple[key1].click_poses[7]/(0.0000001+host_simple[key1].show_poses[7]))
    click_feats1['host_ctrpos_9'].append(host_simple[key1].click_poses[8]/(0.0000001+host_simple[key1].show_poses[8]))
    click_feats1['host_ctrpos_10'].append(host_simple[key1].click_poses[9]/(0.0000001+host_simple[key1].show_poses[9]))
    
    click_feats1['host_pbm'].append(host_simple[key1].total_clicks/(np.sum(host_simple[key1].show_poses*alpha)+0.0000001))
    click_feats1['host_time'].append(host_simple[key1].time)
    click_feats1['host_avg_time'].append(host_simple[key1].time/(host_simple[key1].total_clicks+0.0000001))

100%|██████████| 606050/606050 [00:58<00:00, 10417.95it/s]


In [14]:
with open("click_feats_advanced1.pickle", 'wb') as file:
    pickle.dump(click_feats1, file)

In [15]:
with open("click_feats_advanced2.pickle", 'wb') as file:
    pickle.dump(click_feats2, file)

In [16]:
with open("click_feats_advanced.pickle", 'wb') as file:
    pickle.dump(click_feats, file)
print(16)

16